In [65]:
# Import
import numpy as np
import pandas as pd
import napari

from scipy.interpolate import RBFInterpolator

import tifffile 

from scripts.sample_db import SampleDB
from scripts.config_model import save_experiment_config, tree

import dill


In [43]:
#Upload sample with stck and mask
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Get the sample
exp = sample_db.get_sample('20220426_RM0008_130hpf_fP1_f3')
em_stack_path = exp.paths.em_stack
em_mask_path = exp.paths.em_mask

em_stack =  tifffile.imread(em_stack_path)
em_mask =  tifffile.imread(em_mask_path)
lut = pd.read_csv(exp.paths.em_centroids)


In [44]:
em_centroids= np.array([eval(t) for t in lut['em_centroids_zyx_px']])
print(em_centroids.shape)


(9462, 3)


In [45]:
# Load landmarks to create transform
lmks_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\bigwarp_alignment\20240710_mov_EM_fix_LM_landmarks.csv'

lmks = pd.read_csv(lmks_path, header= None)
# Add the headers as shown in the image
headers = ['Name', 'Active', 'mvg-x', 'mvg-y', 'mvg-z', 'fix-x', 'fix-y', 'fix-z']

# Assign the headers to the DataFrame
lmks.columns = headers

# Saving and updating into exp 
lmks.to_csv(exp.paths.clem_path + "/20240710_mov_EM_fix_LM_landmarks.csv" )
exp.paths.lmks_path = exp.paths.clem_path + "/20240710_mov_EM_fix_LM_landmarks.csv" 
save_experiment_config(exp)

\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\config_20220426_RM0008_130hpf_fP1_f3.json


In [46]:
lmks.head()


,Name,Active,mvg-x,mvg-y,mvg-z,fix-x,fix-y,fix-z
0,Pt-0,False,15.401238,23.064857,67.894922,8.473757,50.299048,48.974068
1,Pt-1,False,16.572585,36.465325,98.828049,35.424851,66.547966,79.553103
2,Pt-2,False,51.741957,17.723752,93.737745,67.471004,42.754333,74.472040
3,Pt-5,False,43.160962,38.318928,39.673012,41.138765,51.146687,6.435591
4,Pt-6,False,38.967385,36.177572,37.005390,35.281416,47.460107,9.680567


In [47]:
em_landmarks = lmks[lmks['Active']==True][['mvg-z','mvg-y','mvg-x']].to_numpy()/[.16,0.4,0.16]
lm_landmarks = lmks[lmks['Active'] == True][['fix-z', 'fix-y', 'fix-x']].to_numpy()/[.4,0.4,0.4]
print(em_landmarks.shape)

(351, 3)


In [48]:
# Setting up the RBFInterpolator for TPS
source_landmarks = em_landmarks
target_landmarks = lm_landmarks
# The 'thin_plate' kernel is used for TPS
tps_kernel = 'thin_plate_spline'

# Instantiate the RBFInterpolator for each dimension
rbf_interpolator = RBFInterpolator(source_landmarks, target_landmarks, kernel=tps_kernel)

In [87]:
# Save
with open(exp.paths.clem_path +'/'+f'{exp.sample.id}_em2lm_interpolator.dill', 'wb') as f:
    dill.dump(rbf_interpolator, f)



In [ ]:
# # Load an existing interpolator 
# with open(exp.paths.clem_path +'/'+f'{exp.sample.id}_em2lm_interpolator.dill', 'rb') as f:
#     loaded_interpolator = dill.load(f)

# # Compare predictions
# test_point = em_landmarks
# original_prediction = rbf_interpolator(test_point)
# loaded_prediction = loaded_interpolator(test_point)
# np.allclose(original_prediction, loaded_prediction)  # Should return True
# 
# # Compare attributes
# original_attributes = rbf_interpolator.__dict__
# loaded_attributes = loaded_interpolator.__dict__
# all(np.array_equal(original_attributes[k], loaded_attributes[k]) 
#     if isinstance(original_attributes[k], np.ndarray)
#     else original_attributes[k] == loaded_attributes[k]
#     for k in original_attributes)  # Should return True

In [70]:
# Apply the transformation using the RBFInterpolator
transformed_landmarks = rbf_interpolator(em_landmarks)
transformed_centroids = rbf_interpolator(em_centroids)
# Save lut 

In [71]:
# Visualize
vv = napari.Viewer()
#landmarks
vv.add_points(em_landmarks, face_color='m')
vv.add_points(lm_landmarks, face_color='y')
vv.add_points(transformed_landmarks , face_color='c')
#centroids
vv.add_points(em_centroids, face_color='r')
vv.add_points(transformed_centroids, face_color='b')

D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Points layer 'transformed_centroids' at 0x20d03ecf340>

In [72]:
lm_stack = tifffile.imread(r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\bigwarp_alignment\lm_stack.tif') 
vv.add_image(lm_stack, name='lm_stack')



<Image layer 'lm_stack' at 0x20d5249fe50>

In [73]:

from scipy.ndimage import map_coordinates
em_mask = tifffile.imread(exp.paths.em_mask)

vv.add_labels(em_mask, name='em_mask')

<Labels layer 'em_mask' at 0x20d5249c430>

In [53]:
# save transformed_em_centroids as lm_stack_processed_centroids
# Convert the numpy array to a list of tuples if it's not already
centroid_list = [tuple(centroid) for centroid in transformed_centroids]

# Create a DataFrame with unique IDs and centroids
lut['lm_stack_processed_centroids_zyx_px'] = centroid_list

lut.head()

,id,em_centroids_zyx_px,lm_stack_processed_centroids_zyx_px
0,1,"(585.8534150215567, 283.09280689811663, 411.51...","(116.81865840534738, 348.3575044800491, 282.45..."
1,2,"(564.9634206623826, 281.9189322787939, 392.601...","(108.89449801383307, 347.8131157787775, 269.19..."
2,3,"(616.4642448512586, 280.7225400457666, 418.734...","(131.9194855128153, 347.20118734227253, 289.86..."
3,4,"(569.6973260003792, 281.62526076237435, 446.58...","(104.54480950737889, 341.5247897065565, 297.01..."
4,5,"(549.7964018525116, 280.40220876380477, 463.83...","(92.86902184159953, 336.6722687165584, 302.118..."


In [59]:
lut_path = str(exp.paths.clem_path +"\\" +exp.sample.id + "_lut_centroids.csv")
lut.to_csv(lut_path, index=False)

exp.paths.lut_path = lut_path
save_experiment_config(exp)

\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\config_20220426_RM0008_130hpf_fP1_f3.json


In [88]:
tree(exp)

sample: Sample
    id: str
        20220426_RM0008_130hpf_fP1_f3
    parents_id: str
        P1
    genotype: str
        aTubulin:GCamp6s x GAD1b:DsRed
    phenotype: str
        positive expression, nacre
    dof: str
        21.04.2022-09:30
    hpf: int
        130
    body_length_mm: int
        4
params_odor: ParamsOdor
    odor_list: list
        ['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA']
    odor_sequence: list
        ['Ala', 'Ser', 'Ctrl', 'TCA', 'GCA', 'TDCA', 'Cad', 'SA', 'Cad', 'TDCA', 'GCA', 'TCA', 'Ser', 'Ctrl', 'Ala', 'SA', 'Ctrl', 'Cad', 'Ala', 'Ser', 'TDCA', 'GCA', 'TCA', 'SA']
    odor_concentration_uM: list
        name: str
            Ala
        concentration_mM: float
            100.0
        name: str
            Ser
        concentration_mM: float
            100.0
        name: str
            Ctrl
        concentration_mM: float
            100.0
        name: str
            TCA
        concentration_mM: float
            10.0
        name: 